checking mismatches between rows and seeing y thats happening

In [7]:
# List of file paths
file_paths = [f"results/new/final_test_predictions_GINConvNet_davis_b3_{i}.csv" for i in range(1, 6)]

df_list = []
for i, path in enumerate(file_paths, start=1):
    df = pd.read_csv(path)
    df.drop(columns=["Fold"], errors="ignore", inplace=True)  # Remove existing Fold column if present
    df["Fold"] = i  # Add new Fold column
    print(f"File {i}: {path} — {len(df)} rows")  # Debug: show number of rows
    print(df.head(1))  # Debug: show sample row to verify Fold is added
    df_list.append(df)
    
    
concatenated_df = pd.concat(df_list, ignore_index=True)

# Save to a new CSV
output_path = "results/new/concatenated_final_test_predictions.csv"
concatenated_df.to_csv(output_path, index=False)

print(f"Saved concatenated DataFrame to {output_path}")

File 1: results/new/final_test_predictions_GINConvNet_davis_b3_1.csv — 5712 rows
    Drug_ID Protein_ID  True_Label  Predicted_Value  Fold
0  10138260      SNARK     6.39794         5.411871     1
File 2: results/new/final_test_predictions_GINConvNet_davis_b3_2.csv — 5508 rows
    Drug_ID Protein_ID  True_Label  Predicted_Value  Fold
0  10427712        TXK         5.0         4.936027     2
File 3: results/new/final_test_predictions_GINConvNet_davis_b3_3.csv — 7072 rows
   Drug_ID Protein_ID  True_Label  Predicted_Value  Fold
0  6450551       GRK1         5.0         4.977173     3
File 4: results/new/final_test_predictions_GINConvNet_davis_b3_4.csv — 6256 rows
    Drug_ID Protein_ID  True_Label  Predicted_Value  Fold
0  11427553       ERK1         5.0         5.762062     4
File 5: results/new/final_test_predictions_GINConvNet_davis_b3_5.csv — 5508 rows
   Drug_ID Protein_ID  True_Label  Predicted_Value  Fold
0   208908      PRKCI         5.0         4.847114     5
Saved concatenated 

In [8]:
import pandas as pd

# Load original train + test
train = pd.read_csv("data/davis_b3_train.csv")
test = pd.read_csv("data/davis_b3_test.csv")
combined = pd.concat([train, test], ignore_index=True)
combined["affinity"] = combined["affinity"].round(5)
original_triples = set(zip(combined["drug_id"], combined["protein_id"], combined["affinity"]))

# Load predictions
# unblinded = pd.read_csv("results/1000EPOCH/crossval_predictions_GINConvNet_davis_b3.csv")
# blinded = pd.read_csv("results/1000EPOCH/concatenated_final_test_predictions.csv")
unblinded = pd.read_csv("results/new/crossval_predictions_GINConvNet_davis_b3.csv")
blinded = pd.read_csv("results/new/concatenated_final_test_predictions.csv")

unblinded["True_Label"] = unblinded["True_Label"].round(5)
blinded["True_Label"] = blinded["True_Label"].round(5)

unblinded_triples = set(zip(unblinded["Drug_ID"], unblinded["Protein_ID"], unblinded["True_Label"]))
blinded_triples = set(zip(blinded["Drug_ID"], blinded["Protein_ID"], blinded["True_Label"]))

# Compare
missing_from_unblinded = original_triples - unblinded_triples
missing_from_blinded = original_triples - blinded_triples

# Report
print(f"✅ Total original triples: {len(original_triples)}")
print(f"✅ Total unblinded prediction triples: {len(unblinded_triples)}")
print(f"✅ Total blinded prediction triples: {len(blinded_triples)}")

print(f"\n❌ Missing from unblinded: {len(missing_from_unblinded)}")
print(f"❌ Missing from blinded: {len(missing_from_blinded)}")

# Optionally preview a few
if missing_from_unblinded:
    print("\nExamples missing from unblinded:", list(missing_from_unblinded)[:5])
if missing_from_blinded:
    print("\nExamples missing from blinded:", list(missing_from_blinded)[:5])


✅ Total original triples: 27621
✅ Total unblinded prediction triples: 27621
✅ Total blinded prediction triples: 27621

❌ Missing from unblinded: 58
❌ Missing from blinded: 58

Examples missing from unblinded: [(10138260, 'LYN', 6.09151), (16722836, 'RSK2(KinDom.1-N-terminal)', 5.09151), (11427553, 'EPHA3', 6.09151), (126565, 'RET(V804M)', 8.17393), (9933475, 'TNIK', 6.09151)]

Examples missing from blinded: [(10138260, 'LYN', 6.09151), (16722836, 'RSK2(KinDom.1-N-terminal)', 5.09151), (11427553, 'EPHA3', 6.09151), (126565, 'RET(V804M)', 8.17393), (9933475, 'TNIK', 6.09151)]
